In [ ]:
from math import sqrt
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from keras import backend as K
from numpy import concatenate
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import requests
import pandas as pd
import sqlite3
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.layers import Dropout
import keras_tuner as kt


# conn =sqlite3.connect('sqlite_8.db')
# c = conn.cursor()

# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(c.fetchall())

# c.execute("SELECT * FROM syslogs ORDER BY timestamp DESC LIMIT 1")
# print(c.fetchall())


limit = '16'
bucket_size = 10
timing =90


variable = int(60*bucket_size)

df = pd.read_pickle("dataframe.gz")


def makeDataframe(df):
    import datetime

    atype = str(df.tail(1)['bucket'].values[0])
    ctype = str(df.head(1)['bucket'].values[0])

    sample_series = []

    end = datetime.datetime.strptime(atype, '%Y-%m-%d %H:%M:%S')
    # start = end - datetime.timedelta(minutes = 3000)
    start = datetime.datetime.strptime(ctype, '%Y-%m-%d %H:%M:%S')

    delta = datetime.timedelta(minutes=bucket_size)
    # start = datetime.datetime.strptime( start, '%Y-%m-%d %H:%M:%S' )
    # end = datetime.datetime.strptime( end, '%Y-%m-%d %H:%M:%S' )
    t = start
    while t <= end :
        sample_series.append(datetime.datetime.strftime( t, '%Y-%m-%d %H:%M:%S'))
        t += delta
    sample = pd.DataFrame(sample_series)
    sample =sample.rename(columns={0:"bucket"})

    sample.insert(1, 'msg_freq',0)

    merged_df = df.merge(sample, how = 'outer', on = ['bucket'])
    new_df = merged_df.sort_values('bucket')
    new_df = new_df.drop(columns = ['msg_freq_y']).fillna(0)

    new_df  = new_df.reset_index()
    new_df = new_df.iloc[:,1:]
    df = new_df.rename(columns={"msg_freq_x": "msg_freq"})
    
    return df



def cumulativeFreq(df):
    df_cs = df.copy()
    df_cs['bucket'] = pd.to_datetime(df_cs['bucket'])
    df_cs["msg_freq"] = df_cs["msg_freq"].iloc[:].groupby([df_cs['bucket'].dt.hour, df_cs['bucket'].dt.date]).cumsum()
    
    return df_cs



def Normalizer(df):
    from sklearn.preprocessing import LabelEncoder
    from sklearn.preprocessing import MinMaxScaler

    df1 = df.iloc[:,1:]   #change required
#     values = dff.values
#     values = values.astype('float32')
#     # normalize features
#     scaler = MinMaxScaler(feature_range=(0, 1))
#     scaled = scaler.fit_transform(values)
#     dff = pd.DataFrame(scaled, columns=df.columns[1:]) #change required
    dff = df1/df1.max()
    dff.insert(0, 'bucket', df["bucket"]) #change required
    
    return dff


def sigmoidDataframe(df):
    
    def sigmoid(x):
        a=0.7
        return 1/(1+5*np.exp(-a*x))
    
    dff = df.copy()
    dff['msg_freq'] = sigmoid(dff['msg_freq'])
    return dff

df = makeDataframe(df)
df_cs = cumulativeFreq(df)

dfn = Normalizer(df)
dfn_cs = Normalizer(df_cs)


Values = df_cs.iloc[:,1].values

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)



# choose a number of time steps; basically the memory of the network
n_steps = int(timing/bucket_size)

# split into samples
X, y = split_sequence(Values, n_steps)

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))


def wisesplit(num,div):
    
    part = num - div
    while part/num > 0.8:
        part = part-div
    
    return part

def print_factors(num):
    import numpy as np
    factors = []
    for i in range(1, num + 1):
        if num % i == 0:
            factors.append(i)
    return np.array(factors)

arr =print_factors(len(X))
div = arr[int(0.6*arr.shape[0])]

split = wisesplit(len(X),div)
# split =int(0.8*len(X))
# split = -272

train_X = np.array(X)[:split]
train_y = np.array(y)[:split]
test_X = np.array(X)[split:]
test_y = np.array(y)[split:]

train_y =train_y.reshape((train_y.shape[0],1))

test_y = test_y.reshape((test_y.shape[0],1))

def compute_hcf(x, y):

# choose the smaller number
    if x > y:
        smaller = y
    else:
        smaller = x
    for i in range(1, smaller+1):
        if((x % i == 0) and (y % i == 0)):
            hcf = i 
    return hcf

batchsize = compute_hcf(train_X.shape[0],test_X.shape[0])
batch_arr = print_factors(batchsize)
bt = batch_arr[-1]

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

    
    
def build_model(hp):
    model = Sequential()
    model.add(LSTM(hp.Int('input_unit',min_value=32,max_value=128, step =32),return_sequences=False, batch_input_shape=(bt,train_X.shape[1],train_X.shape[2]), stateful=True))
#     for i in range(hp.Int('n_layers', 1, 4)):
#         model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=96,step=32),return_sequences=True))
#     model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=128,step=32)))
#     model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    hp_loss = hp.Choice('loss', values=['mean_squared_logarithmic_error'])
    model.add(Dense(train_y.shape[1]))
    model.compile(loss=hp_loss, optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=0.0001,max_value=0.001, step =0.0001)))
    return model

tuner = kt.Hyperband(build_model,
                     objective='loss',
                     max_epochs=1500,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')


stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


print(tuner.search(
        x=train_X,
        y=train_y,
        epochs=1500,
        batch_size = bt,
        validation_data=(test_X,test_y)
))

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

best_model = tuner.get_best_models()[0]

print("The optimal number of units in the first densely-connected", best_hps.get('input_unit'))
print("The optimal learning rate for the optimizer is ",best_hps.get('learning_rate') )
print("The best loss is ", best_hps.get('loss'))



# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(x=train_X, y=train_y, epochs=1500, validation_data=(test_X,test_y), batch_size=batchsize, verbose =0, shuffle=False)

loss_per_epoch = history.history['loss']
best_epoch = loss_per_epoch.index(min(loss_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
final_history = hypermodel.fit(x=train_X, y=train_y, epochs=best_epoch, validation_data=(test_X,test_y), batch_size=batchsize, verbose =1, shuffle=False)
hyper_pred= hypermodel.predict(test_X,bt)

#PLOTTING

plt.figure(figsize=(12,8))   
plt.plot(final_history.history['loss'],label='Training loss')
plt.plot(final_history.history['val_loss'], label ="Validation loss")
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
#             plt.ylim(0,0.2)
plt.legend(['Training loss', 'Validation loss'])
plt.savefig('bestmodel_loss_10_90_layer1.pdf', dpi=250)
plt.show()

print(hypermodel.summary())
hyper_pred_df = pd.DataFrame(hyper_pred.flatten())
test_anom = pd.DataFrame(test_y)

fig = go.Figure()    
fig.add_trace(go.Scatter(x=df_cs['bucket'][split+n_steps:], y= test_anom[0], name='actual'))
# fig.add_trace(go.Scatter(x=df_cs['bucket'][split+n_steps:], y= tuner_pred[0], name='tuner_prediction'))
# fig.add_trace(go.Scatter(x=df_cs['bucket'][split+n_steps:], y= pred_df[0], name= 'model_prediction'))
fig.add_trace(go.Scatter(x=df_cs['bucket'][split+n_steps:], y= hyper_pred_df[0], name='hypermodel_prediction'))
fig.update_layout(showlegend=True, title='Frequency v/s Time',width=1600, height=800)
fig.update_yaxes(title_text='Normalized frequency')
fig.update_xaxes(title_text='Time')
fig.update_layout(xaxis_range=['2022-01-09 12:00:00','2022-01-10 00:00:00'])
# fig.update_layout(yaxis_range=[-0.1,1])
fig.show()
fig.write_image("bestmodel_predictions_10_90_layer1.pdf")